# Elevation Interpolation and Accuracy Assessment

Logan Gall

Attributions: ChatGPT, Laure Briol

In [2]:
#imports
import arcpy
import psycopg2
from psycopg2 import sql
import os
import random

#location to current directory
file_path = os.path.dirname(arcpy.mp.ArcGISProject('CURRENT').filePath)
os.chdir(file_path)
#absolute Path for geodatabase
arcpy.env.workspace = file_path

## Database Connection

In [3]:
#Database connection
conn = psycopg2.connect(
    dbname="gis5572",
    user="postgres",
    password="Passwordd",
    host="35.188.97.184",
    port="5432"
)
cur = conn.cursor()

In [4]:
#Select the table for our elevation data. Return shape as WKT
sql_query = """
SELECT grid_code, ST_AsText(shape) AS WKT
FROM dem_resampled_point;
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch the results
fetched_data = cur.fetchall()

print(len(fetched_data))

# Close the cursor and connection
cur.close()
conn.close()

2703


In [ ]:
output_path = "5572Lab3.gdb"  #Output geodatabase
feature_class_name = "ElevationPointRaw" #Output data
geometry_type = "POINT"

#Spatial reference
spatial_reference = arcpy.SpatialReference(26915)

#Create empty feature class
arcpy.CreateFeatureclass_management(output_path, feature_class_name, geometry_type, spatial_reference=spatial_reference)

#Add fields to the feature class
arcpy.AddField_management(f"{output_path}/{feature_class_name}", "elevation", "LONG")

#Insert data into the feature class
with arcpy.da.InsertCursor(f"{output_path}/{feature_class_name}", ["elevation", "SHAPE@WKT"]) as cursor:
    for row in fetched_data:
        #Inserts data assuming SQL return is in same order
        cursor.insertRow(row)

print("Data transfer to Feature Class is complete.")

In [6]:
# Set up Database Connection within Arc
# Parameters for the database connection
out_folder = arcpy.env.workspace + '\\database_connection'  # Folder where the .sde file will be stored
out_nam = "postgres_connection.sde"  # Name of the .sde file to create

# Database connection properties
server = "35.188.97.184"  # Name or IP of the PostgreSQL server
service = "5432"  # PostgreSQL port (default is 5432)
database = "gis5572"  # Name of the database
username = "postgres"
password = "Passwordd"
authentication_type = "DATABASE_AUTH"
version = "sde.DEFAULT"

#Create database connection file
arcpy.management.CreateDatabaseConnection(
    out_folder_path=out_folder,
    out_name=out_nam,
    database_platform="POSTGRESQL",
    instance=server,
    account_authentication="DATABASE_AUTH",
    username=username,
    password=password,
    save_user_pass="SAVE_USERNAME",
    database="gis5572",
    schema="",
    version_type="TRANSACTIONAL",
    version="",
    date=None,
    auth_type="",
    project_id="",
    default_dataset="",
    refresh_token='',
    key_file=None,
    role="",
    warehouse="",
    advanced_options=""
)

<Result 'C:\\Users\\Logan\\Documents\\ArcGIS\\Projects\\5572Lab3\\database_connection\\postgres_connection.sde'>

## Sampling

### Justification for Train/Test Split

Using a larger dataset for training, like in a 90/10 split, helps with statistical significance. The more data we have to train the model, the more confident we can be about the patterns it learns and the predictions it makes. The larger training set allows the model to better understand and generalize the complex patterns in elevation data, making its predictions reliable. The large training set helps make it robust to noise and outliers. With 90% of the data used for training, the model has a better chance of identifying and learning to ignore or properly interpret significant changes in elevation.

In [8]:
#path to feature class
feature_class_path = "\\5572Lab3.gdb\\ElevationPointRaw"

#Output path for the subset feature class
output_feature_class = "\\5572Lab3.gdb\\ElevationPoint10" #Testing data
output_feature_class2 = "\\5572Lab3.gdb\\ElevationPoint90" #Training data


#Get the count of rows
feature_count = arcpy.GetCount_management(feature_class_path)[0]

#Calculate 10% of the total features
subset_count = int(float(feature_count) * 0.1)

#Generate a list of random indices
random.seed = 0
#list selecting points to be selected (list of objectID's)
random_indices = random.sample(range(int(feature_count)), subset_count)

#create a query to select these indices
oids = [str(oid) for oid in random_indices]
query = f"OBJECTID IN ({','.join(oids)})"
query2 = f"OBJECTID NOT IN ({','.join(oids)})"

#Select_analysis to create a new feature class with the subset
arcpy.Select_analysis(feature_class_path, output_feature_class, query)
arcpy.Select_analysis(feature_class_path, output_feature_class2, query2)

print(f"Created subset feature classes")

Created subset feature classes


## Leave one out interpolation analysis

In [9]:
#Leave one out exploratory analysis
## Compare Ordinary Kriging, Universal Kriging, and IDW
## Uses training data to perform analysis
arcpy.ga.ExploratoryInterpolation(
    in_features="ElevationPoint90",
    value_field="elevation",
    out_cv_table=arcpy.env.workspace + "\\5572Lab3.gdb\\InterpolationExp",
    out_geostat_layer=None,
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'C:\\Users\\Logan\\Documents\\ArcGIS\\Projects\\5572Lab3\\5572Lab3.gdb\\InterpolationExp'>

In [10]:
#Push data to remote database
arcpy.conversion.TableToGeodatabase(Input_Table=arcpy.env.workspace + "\\5572Lab3.gdb\\InterpolationExp", Output_Geodatabase=arcpy.env.workspace + "\\database_connection\\postgres_connection.sde")

<Result 'C:\\Users\\Logan\\Documents\\ArcGIS\\Projects\\5572Lab3\\database_connection\\postgres_connection.sde'>

## Perform Interpolations

In [56]:
## Create 3 maps of our 3 interpolation algorithms
## IDW
### Deterministic simple interpolation, baseline
arcpy.ga.IDW(
    in_features="ElevationPoint90",
    z_field="elevation",
    out_ga_layer=None,
    out_raster=arcpy.env.workspace + "\\5572Lab3.gdb\\Elevation_IDW",
    cell_size=9000,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=212324.191980094 S_MINOR=212324.191980094 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

## Ordinary Kriging
### Simplified model with few assumptions
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace + "\\5572Lab3.gdb"):
    Elevation_OrdinaryKriging = arcpy.sa.Kriging(
        in_point_features="ElevationPoint90",
        z_field="elevation",
        kriging_model="Spherical # # # #",
        cell_size=9000,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    Elevation_OrdinaryKriging.save(arcpy.env.workspace + "\\5572Lab3.gdb\\Elevation_OrdinaryKriging")
    
## Universal Kriging
### Models the underlying data, more complex
### This has highest accuracy in our model
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace + "\\5572Lab3.gdb"):
    Elevation_UniversalKriging = arcpy.sa.Kriging(
        in_point_features="ElevationPoint90",
        z_field="elevation",
        kriging_model="LinearDrift 2196.000000 # # #",
        cell_size=9000,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    Elevation_UniversalKriging.save(arcpy.env.workspace + "\\5572Lab3.gdb\\Elevation_UniversalKriging")

In [12]:
#Push to database
##Function to convert data to point, then push the point data to the database
def point_and_push(data_name, local_gdb_path, geodatabase_path):
    
    #Convert raster to point data (shape)
    arcpy.conversion.RasterToPoint(
        in_raster=local_gdb_path + '\\' + data_name,
        out_point_features=local_gdb_path + '\\' + data_name + '_point',
        raster_field="Value"
    )

    print(data_name + ' converted to point')
    
    #Push data to remote database
    arcpy.conversion.FeatureClassToGeodatabase(local_gdb_path + '\\' + data_name + '_point', geodatabase_path)
    print(local_gdb_path + '\\' + data_name + '_point' + ' pushed to database')

In [13]:
#Pushing three interpolation algorithms to database
local_gdb_path = arcpy.env.workspace + "\\5572Lab3.gdb"
#Remote database file
geodatabase_path = arcpy.env.workspace + "\\database_connection\\postgres_connection.sde"

data_name = "Elevation_IDW"
point_and_push(data_name, local_gdb_path, geodatabase_path)

data_name = "Elevation_OrdinaryKriging"
point_and_push(data_name, local_gdb_path, geodatabase_path)

data_name = "Elevation_UniversalKriging"
point_and_push(data_name, local_gdb_path, geodatabase_path)

Elevation_IDW converted to point
C:\Users\Logan\Documents\ArcGIS\Projects\5572Lab3\5572Lab3.gdb\Elevation_IDW_point pushed to database
Elevation_OrdinaryKriging converted to point
C:\Users\Logan\Documents\ArcGIS\Projects\5572Lab3\5572Lab3.gdb\Elevation_OrdinaryKriging_point pushed to database
Elevation_UniversalKriging converted to point
C:\Users\Logan\Documents\ArcGIS\Projects\5572Lab3\5572Lab3.gdb\Elevation_UniversalKriging_point pushed to database


# Accuracy Assessment

Perform accuracy assessment on the testing 10% of data compared to our most accuracte interpolation

In [1]:
#Our testing data (10% sample)
input_point_dataset = arcpy.env.workspace + "\\5572Lab3.gdb\\ElevationPoint10"
#Best interpolation
input_raster = arcpy.env.workspace + "\\5572Lab3.gdb\\Elevation_UniversalKriging"
#Store results
output_point_dataset = arcpy.env.workspace + "\\5572Lab3.gdb\\Elevation_Accuracy"

#Copy the point dataset
arcpy.CopyFeatures_management(input_point_dataset, output_point_dataset)

#Pull raster values to the copied points dataset
#Name of field in interpolated raster data that we are pulling from
extracted_field_name = "VALUE"
#Pulling these values into the copied dataset
arcpy.sa.ExtractValuesToPoints(input_point_dataset, input_raster, output_point_dataset,
                               "INTERPOLATE", "VALUE_ONLY")

#Names of columns
## elevation (the testing data)
## RASTERVALU (The interpolation data we pulled)
## Diff_Value (new field that is elevation minus RASTERVALU)
point_value_field = "elevation"
difference_field = "Diff_Value"
extracted_field_name = "RASTERVALU"

#Add a new field for the difference
arcpy.management.CalculateField(
    in_table="Elevation_Accuracy",
    field="Diff_Value",
    expression="!elevation!-!RASTERVALU!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

print("Process completed.")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset C:\Users\Logan\Documents\ArcGIS\Projects\5572Lab3\5572Lab3.gdb\5572Lab3.gdb\ElevationPoint10 does not exist or is not supported
Failed to execute (CopyFeatures).


In [15]:
#push elevation_accuracy to database    
arcpy.conversion.FeatureClassToGeodatabase(arcpy.env.workspace + "\\5572Lab3.gdb\\Elevation_Accuracy", arcpy.env.workspace + "\\database_connection\\postgres_connection.sde")
print('pushed to database')

pushed to database
